In [1]:
import pandas as pd
import itertools
import os
import pandas.io.formats.style

In [2]:
pd.options.display.float_format = "{:,.3f}".format

### Globals, Helper Functions, etc

In [3]:
MEASURE_NAME_DICT = {
    "AlignedCosineSimilarity": "AlignCos",
    "CKA": "CKA",
    "ConcentricityDifference": "ConcDiff",
    "DistanceCorrelation": "DistCorr",
    "EigenspaceOverlapScore": "EOS",
    "GeometryScore": "GS",
    "Gulp": "GULP",
    "HardCorrelationMatch": "HardCorr",
    "IMDScore": "IMD",
    "JaccardSimilarity": "Jaccard",
    "LinearRegression": "LinReg",
    "MagnitudeDifference": "MagDiff",
    "OrthogonalAngularShapeMetricCentered": "ShapeMet",
    "OrthogonalProcrustesCenteredAndNormalized": "OrthProc",
    "PWCCA": "PWCCA",
    "PermutationProcrustes": "PermProc",
    "ProcrustesSizeAndShapeDistance": "ProcDist",
    "RSA": "RSA",
    "RSMNormDifference": "RSMDiff",
    "RankSimilarity": "RankSim",
    "SVCCA": "SVCCA",
    "SecondOrderCosineSimilarity": "2nd-Cos",
    "SoftCorrelationMatch": "SoftCorr",
    "UniformityDifference": "UnifDiff",
}

PIVOT_COL_DICT = {
    "cora": "Cora",
    "flickr": "Flickr",
    "ogbn-arxiv": "OGBN-Arxiv",
    "GraphSAGE": "SAGE",
    "violation_rate": "Violation Rate",
    "correlation": "Spearman Correlation"
}

COLUMN_NAME_DICT = {
    "similarity_measure": "Similarity Measure", 
    "quality_measure": "Measure",
    "functional_similarity_measure": "Measure",
    "architecture": "Model",
    "representation_dataset": "Dataset"
}

LATEX_FORMAT_DICT = {
    "hrules": True,
    "column_format": "l||rrr|rrr|rrr||rrr|rrr|rrr",
    "multicol_align": "c",
}

In [26]:
# EXPERIMENT_RESULTS_PATH = 'C:/Users/Tobias/Eigene Dokumente/Research/similaritybench/experiments/results/res_csvs/'
EXPERIMENT_RESULTS_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "experiments", "results", "res_csvs")
COMPARISON_TYPE_DICT = {
    "label_test": "group_separation",
    "layer_test": "monotonicity",
    "augmentation_test": "group_separation",
    "shortcut_test": "group_separation"
}

EXPERIMENTS = list(COMPARISON_TYPE_DICT.keys())
DATASETS = ["cora", "flickr", "ogbn-arxiv"]
ARCHITECTURES = ["GCN", "GraphSAGE", "GAT"]

def FULL_DF_FILE_NAME(experiment, comparison_type, dataset, groups=5):
    if groups < 5:
        return f"{experiment}_{comparison_type}_{dataset}_{groups}groups_full.csv"
    return f"{experiment}_{comparison_type}_{dataset}_full.csv"

In [5]:
def get_pivot_table(experiment, dataset):
    path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset))
    
    df = pd.read_csv(path)
    data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]]
    return data.pivot(index="similarity_measure", columns=["representation_dataset", "architecture", "quality_measure"], values="value")

In [18]:
# def get_agg_pivot_table(experiment, datasets, groups=5):
#     dfs = []
#     for dataset in datasets:
#         path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset, groups))
#         df = pd.read_csv(path)
#         data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]].dropna()
   
#         data = data.rename(COLUMN_NAME_DICT, axis="columns")
#         dfs.append(data.iloc[:])
#     df_cc = pd.concat(dfs, axis=0)
#     df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="value", aggfunc="mean")
#     df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
#     return df_res.rename(PIVOT_COL_DICT, axis="columns")

In [58]:
def get_agg_pivot_table(experiment, datasets, conformity=True):
    dfs = []
    full_csvs = os.listdir(EXPERIMENT_RESULTS_PATH)
    
    for dataset in datasets:
        for f in full_csvs:
            if experiment in f and dataset in f:
                df = pd.read_csv(os.path.join(EXPERIMENT_RESULTS_PATH, f), encoding="utf-8")
                data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]].dropna()
                data = data.rename(COLUMN_NAME_DICT, axis="columns")
                dfs.append(data.iloc[:])
    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="value", aggfunc="mean")
    if conformity:
        df_res["violation_rate"] = 1 - df_res["violation_rate"]
        df_res = df_res.rename({"violation_rate": "Conformity Rate"}, axis=1)
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    return df_res.rename(PIVOT_COL_DICT, axis="columns")

### Results for Group Separation Experiments

In [59]:
layer_cora_df = get_agg_pivot_table("layer_test", ["cora"])
styled = pd.io.formats.style.Styler(
    layer_cora_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("cora_results_layer_test.tex", **LATEX_FORMAT_DICT)
layer_cora_df

Measure            Spearman Correlation                   Conformity Rate  \
Dataset                            Cora                              Cora   
Model                               GAT   GCN  SAGE  PGNN             GAT   
Similarity Measure                                                          
AlignCos                          1.000 0.980 1.000 0.313           1.000   
CKA                               1.000 0.980 1.000 1.000           1.000   
ConcDiff                          0.403 0.850 0.250 0.117           0.620   
DistCorr                          1.000 1.000 1.000 1.000           1.000   
EOS                               1.000 1.000 1.000 0.987           1.000   
GULP                              1.000 1.000 1.000 0.720           1.000   
HardCorr                          0.540 0.833 0.983 0.777           0.760   
IMD                               1.000 1.000 1.000 0.887           1.000   
Jaccard                           1.000 1.000 1.000 1.000           1.000   
LinReg                            1.000 1.000 1.000 1.000           1.000   
MagDiff                           0.893 0.550 0.490 0.910           0.920   
ShapeMet                          1.000 1.000 1.000 0.910           1.000   
OrthProc                          1.000 1.000 1.000 0.910           1.000   
PWCCA                             1.000 1.000 1.000 1.000           1.000   
PermProc                          1.000 0.917 1.000 1.000           1.000   
ProcDist                          1.000 0.993 1.000 0.910           1.000   
RSA                               1.000 0.837 0.910 0.607           1.000   
RSMDiff                           1.000 0.993 1.000 0.910           1.000   
RankSim                           1.000 1.000 1.000 1.000           1.000   
SVCCA                             0.613 0.410 0.777 1.000           0.810   
2nd-Cos                           1.000 1.000 1.000 0.900           1.000   
SoftCorr                          0.500 0.957 0.993 0.753           0.780   
UnifDiff                          0.590 0.667 0.667 0.597           0.690   

Measure                               
Dataset                               
Model                GCN  SAGE  PGNN  
Similarity Measure                    
AlignCos           0.970 1.000 0.880  
CKA                0.980 1.000 1.000  
ConcDiff           0.850 0.520 0.510  
DistCorr           1.000 1.000 1.000  
EOS                1.000 1.000 0.980  
GULP               1.000 1.000 0.850  
HardCorr           0.910 0.990 0.820  
IMD                1.000 1.000 0.950  
Jaccard            1.000 1.000 1.000  
LinReg             1.000 1.000 1.000  
MagDiff            0.630 0.720 0.970  
ShapeMet           1.000 1.000 0.970  
OrthProc           1.000 1.000 0.970  
PWCCA              1.000 1.000 0.980  
PermProc           0.910 1.000 1.000  
ProcDist           0.990 1.000 0.970  
RSA                0.900 0.970 0.740  
RSMDiff            0.990 1.000 0.970  
RankSim            1.000 1.000 1.000  
SVCCA              0.630 0.800 1.000  
2nd-Cos            1.000 1.000 0.900  
SoftCorr           0.950 0.990 0.820  
UnifDiff           0.700 0.700 0.710

#### Random Label Test Results

In [37]:
get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.433 0.457 0.479  0.286 0.839 0.423      0.931 0.981   
CKA                0.424 0.427 0.424  0.270 0.733 0.657      1.000 1.000   
ConcDiff           0.394 0.203 0.391  0.208 0.366 0.570      0.997 0.964   
DistCorr           0.424 0.431 0.444  0.217 0.860 0.427      1.000 1.000   
EOS                0.284 0.295 0.259  0.219 0.413 0.424      0.340 0.253   
GULP               0.285 0.276 0.260  0.227 0.192 0.424      0.303 0.302   
HardCorr           0.424 0.425 0.424  0.332 0.771 0.457      0.544 0.828   
IMD                0.815 0.737 0.975  0.227 0.234 0.335      0.882 1.000   
Jaccard            0.427 0.425 0.424  0.292 0.564 0.430      0.778 0.827   
LinReg             0.456 0.492 0.431  0.230 0.223 0.449      0.467 0.444   
MagDiff            0.370 0.199 0.243  0.185 0.661 0.718      0.343 0.551   
ShapeMet           0.424 0.426 0.424  0.275 0.882 0.434      0.767 0.826   
OrthProc           0.424 0.426 0.424  0.275 0.882 0.434      0.767 0.826   
PermProc           0.435 0.443 0.386  0.192 0.772 0.904      0.931 0.678   
ProcDist           0.433 0.452 0.451  0.190 0.785 0.625      1.000 0.915   
RSA                0.425 0.457 0.435  0.334 0.737 0.424      0.490 0.961   
RSMDiff            0.779 0.518 0.527  0.189 0.715 0.915      1.000 0.997   
RankSim            0.475 0.452 0.423  0.333 0.470 0.426      0.779 0.844   
SVCCA              0.422 0.347 0.306  0.268 0.332 0.796      0.932 1.000   
2nd-Cos            0.733 0.559 0.735  0.365 0.611 0.423      0.950 0.994   
SoftCorr           0.423 0.425 0.424  0.332 0.603 0.449      0.547 0.828   
UnifDiff           0.320 0.334 0.497  0.396 0.531 0.902      0.334 0.531   

Measure                  Conformity Rate                                 \
Dataset                             Cora             Flickr               
Model               SAGE             GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                        
AlignCos           0.699           0.601 0.659 0.664  0.579 0.936 0.500   
CKA                1.000           0.507 0.549 0.512  0.542 0.907 0.861   
ConcDiff           1.000           0.739 0.545 0.781  0.481 0.723 0.848   
DistCorr           1.000           0.521 0.590 0.631  0.518 0.948 0.558   
EOS                0.426           0.658 0.649 0.551  0.581 0.500 0.516   
GULP               0.426           0.659 0.631 0.556  0.571 0.503 0.512   
HardCorr           0.828           0.508 0.527 0.511  0.667 0.941 0.679   
IMD                1.000           0.957 0.929 0.993  0.537 0.467 0.693   
Jaccard            0.432           0.553 0.527 0.508  0.568 0.771 0.579   
LinReg             0.677           0.688 0.713 0.579  0.516 0.489 0.663   
MagDiff            0.490           0.737 0.512 0.604  0.472 0.892 0.934   
ShapeMet           0.726           0.522 0.539 0.513  0.533 0.952 0.604   
OrthProc           0.726           0.522 0.539 0.513  0.533 0.952 0.604   
PermProc           0.724           0.609 0.645 0.714  0.500 0.939 0.969   
ProcDist           0.983           0.597 0.673 0.677  0.510 0.932 0.882   
RSA                0.430           0.539 0.693 0.615  0.637 0.885 0.525   
RSMDiff            1.000           0.863 0.732 0.831  0.509 0.911 0.971   
RankSim            0.552           0.643 0.655 0.535  0.583 0.663 0.530   
SVCCA              1.000           0.591 0.619 0.555  0.543 0.689 0.913   
2nd-Cos            0.960           0.864 0.799 0.869  0.660 0.823 0.501   
SoftCorr           0.818           0.505 0.532 0.511  0.649 0.855 0.671   
UnifDiff           0.539           0.672 0.695 0.815  0.749 0.840 0.963   

Measure                                    
Dataset            OGBN-Arxi

In [54]:
label_test_df = get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    label_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_label_test.tex", **LATEX_FORMAT_DICT)

#### Shortcut Test Results

In [56]:
get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.303 0.319 0.599  0.480 0.895 1.000      1.000 1.000   
CKA                0.463 0.273 0.777  0.330 0.282 1.000      1.000 1.000   
ConcDiff           0.204 0.182 0.169  0.324 0.182 0.176      1.000 0.815   
DistCorr           0.459 0.284 0.820  0.317 0.333 1.000      1.000 1.000   
EOS                0.163 0.217 0.495  0.544 0.461 0.433      0.968 1.000   
GULP               0.172 0.197 0.496  0.451 0.196 0.433      0.965 0.986   
HardCorr           0.219 0.200 0.347  0.519 0.549 1.000      0.835 0.798   
IMD                0.601 0.732 0.793  0.358 0.759 0.967      0.923 0.609   
Jaccard            0.380 0.305 0.782  0.545 1.000 0.828      1.000 1.000   
LinReg             0.334 0.299 0.742  0.357 0.365 0.609      1.000 1.000   
MagDiff            0.187 0.267 0.151  0.198 0.266 0.783      1.000 0.550   
ShapeMet           0.366 0.274 0.794  0.307 0.581 1.000      1.000 1.000   
OrthProc           0.366 0.274 0.794  0.307 0.581 1.000      1.000 1.000   
PermProc           0.225 0.207 0.268  0.272 0.418 1.000      0.766 0.649   
ProcDist           0.370 0.247 0.812  0.331 0.599 1.000      1.000 1.000   
RSA                0.578 0.299 0.432  0.521 0.894 1.000      0.902 0.966   
RSMDiff            0.689 0.462 0.497  0.362 0.476 0.920      1.000 1.000   
RankSim            0.362 0.332 0.645  0.538 1.000 0.774      1.000 1.000   
SVCCA              0.571 0.191 0.357  0.315 0.239 0.929      0.825 1.000   
2nd-Cos            0.472 0.343 0.907  0.522 1.000 1.000      1.000 1.000   
SoftCorr           0.218 0.209 0.404  0.571 0.663 1.000      0.830 0.816   
UnifDiff           0.188 0.192 0.637  0.480 0.593 0.499      0.974 0.991   

Measure                  Conformity Rate                                 \
Dataset                             Cora             Flickr               
Model               SAGE             GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                        
AlignCos           1.000           0.589 0.695 0.881  0.769 0.981 1.000   
CKA                0.981           0.636 0.661 0.903  0.652 0.572 1.000   
ConcDiff           0.962           0.498 0.468 0.427  0.607 0.497 0.463   
DistCorr           0.994           0.622 0.688 0.916  0.718 0.687 1.000   
EOS                0.724           0.425 0.574 0.740  0.809 0.872 0.600   
GULP               0.724           0.439 0.525 0.739  0.811 0.492 0.600   
HardCorr           0.724           0.581 0.559 0.768  0.808 0.829 1.000   
IMD                0.920           0.880 0.928 0.957  0.622 0.943 0.993   
Jaccard            1.000           0.714 0.697 0.929  0.840 1.000 0.963   
LinReg             1.000           0.624 0.617 0.947  0.723 0.703 0.811   
MagDiff            0.535           0.489 0.481 0.415  0.477 0.669 0.886   
ShapeMet           1.000           0.620 0.675 0.915  0.721 0.798 1.000   
OrthProc           1.000           0.620 0.675 0.915  0.721 0.798 1.000   
PermProc           0.433           0.519 0.588 0.672  0.576 0.719 1.000   
ProcDist           0.997           0.604 0.645 0.913  0.608 0.815 1.000   
RSA                0.978           0.775 0.702 0.784  0.813 0.970 1.000   
RSMDiff            1.000           0.909 0.697 0.757  0.595 0.749 0.967   
RankSim            1.000           0.723 0.655 0.895  0.830 1.000 0.922   
SVCCA              0.970           0.699 0.462 0.597  0.663 0.567 0.973   
2nd-Cos            1.000           0.689 0.744 0.971  0.821 1.000 1.000   
SoftCorr           0.724           0.579 0.570 0.789  0.824 0.867 1.000   
UnifDiff           0.717           0.475 0.518 0.911  0.673 0.831 0.766   

Measure                                    
Dataset            OGBN-Arxi

In [57]:
shortcut_test_df = get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    shortcut_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_shortcut_test.tex", **LATEX_FORMAT_DICT)

#### Augmentation Test Results

In [58]:
get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.908 0.510 0.635  0.544 0.686 0.698      1.000 0.736   
CKA                0.906 0.697 0.731  0.508 0.572 0.747      0.987 0.997   
ConcDiff           0.623 0.174 0.504  0.534 0.412 0.348      0.508 0.426   
DistCorr           0.891 0.785 0.732  0.609 0.602 0.794      0.989 1.000   
EOS                0.342 0.535 0.618  0.459 0.679 0.529      1.000 0.819   
GULP               0.349 0.448 0.612  0.555 0.223 0.542      1.000 0.557   
HardCorr           0.513 0.512 0.633  0.571 0.712 0.716      0.536 0.466   
IMD                0.238 0.711 0.347  0.281 0.590 0.586      1.000 1.000   
Jaccard            0.953 0.781 0.807  0.580 0.743 0.880      1.000 1.000   
LinReg             0.872 0.943 0.850  0.338 0.302 0.805      0.809 0.724   
MagDiff            0.535 0.204 0.390  0.430 0.588 0.174      0.693 0.771   
ShapeMet           0.835 0.812 0.784  0.540 0.627 0.760      0.744 0.724   
OrthProc           0.835 0.812 0.784  0.540 0.627 0.760      0.744 0.724   
PermProc           0.423 0.701 0.613  0.398 0.649 0.685      0.936 0.881   
ProcDist           0.749 0.764 0.693  0.401 0.696 0.814      1.000 1.000   
RSA                0.784 0.385 0.573  0.586 0.680 0.718      0.674 0.745   
RSMDiff            1.000 0.987 0.795  0.401 0.644 0.933      1.000 1.000   
RankSim            0.947 0.540 0.618  0.553 0.746 0.882      1.000 1.000   
SVCCA              0.339 0.267 0.294  0.495 0.643 0.673      0.820 0.889   
2nd-Cos            0.967 0.734 0.863  0.622 0.777 0.920      1.000 1.000   
SoftCorr           0.496 0.630 0.702  0.546 0.729 0.581      0.448 0.425   
UnifDiff           1.000 0.747 0.534  0.247 0.683 0.239      0.447 0.757   

Measure                  Conformity Rate                                 \
Dataset                             Cora             Flickr               
Model               SAGE             GAT   GCN  SAGE    GAT   GCN  SAGE   
Similarity Measure                                                        
AlignCos           0.427           0.978 0.783 0.881  0.808 0.870 0.901   
CKA                1.000           0.965 0.897 0.919  0.736 0.827 0.912   
ConcDiff           0.533           0.867 0.492 0.781  0.742 0.779 0.681   
DistCorr           1.000           0.965 0.929 0.912  0.812 0.838 0.941   
EOS                0.488           0.752 0.857 0.896  0.703 0.875 0.797   
GULP               0.484           0.763 0.790 0.891  0.839 0.523 0.809   
HardCorr           0.506           0.805 0.849 0.895  0.851 0.894 0.916   
IMD                1.000           0.470 0.929 0.640  0.531 0.873 0.866   
Jaccard            0.991           0.984 0.904 0.946  0.841 0.885 0.973   
LinReg             0.724           0.952 0.983 0.963  0.691 0.527 0.933   
MagDiff            1.000           0.827 0.515 0.742  0.659 0.868 0.449   
ShapeMet           0.724           0.952 0.950 0.931  0.768 0.848 0.937   
OrthProc           0.724           0.952 0.950 0.931  0.768 0.848 0.937   
PermProc           0.734           0.514 0.780 0.887  0.616 0.872 0.901   
ProcDist           1.000           0.897 0.885 0.905  0.626 0.887 0.949   
RSA                0.495           0.938 0.681 0.844  0.841 0.857 0.920   
RSMDiff            1.000           1.000 0.997 0.929  0.620 0.873 0.969   
RankSim            1.000           0.983 0.722 0.880  0.835 0.889 0.961   
SVCCA              0.813           0.579 0.543 0.551  0.684 0.838 0.864   
2nd-Cos            0.992           0.989 0.875 0.972  0.836 0.908 0.981   
SoftCorr           0.460           0.796 0.886 0.923  0.837 0.872 0.852   
UnifDiff           0.526           1.000 0.911 0.859  0.659 0.903 0.518   

Measure                                    
Dataset            OGBN-Arxi

In [60]:
augment_test_df = get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    augment_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_augmentation_test.tex", **LATEX_FORMAT_DICT)

#### Layer Monotonicity Test Results

In [61]:
layer_test_df = get_agg_pivot_table("layer_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    layer_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_layer_test.tex", **LATEX_FORMAT_DICT)

In [62]:
layer_test_df

Measure            Spearman Correlation                                  \
Dataset                            Cora             Flickr                
Model                               GAT   GCN  SAGE    GAT   GCN   SAGE   
Similarity Measure                                                        
AlignCos                          1.000 0.980 1.000  0.040 0.327  0.680   
CKA                               1.000 0.980 1.000  0.387 0.603  0.890   
ConcDiff                          0.403 0.850 0.250  0.243 0.383 -0.273   
DistCorr                          1.000 1.000 1.000  0.467 0.630  0.993   
EOS                               1.000 1.000 1.000  0.917 1.000  1.000   
GULP                              1.000 1.000 1.000  0.330 0.540  1.000   
HardCorr                          0.540 0.833 0.983  0.213 0.627  0.803   
IMD                               1.000 1.000 1.000  0.970 1.000  0.820   
Jaccard                           1.000 1.000 1.000  0.953 0.950  0.973   
LinReg                            1.000 1.000 1.000  0.030 0.450  1.000   
MagDiff                           0.893 0.550 0.490  0.383 0.583  0.500   
ShapeMet                          1.000 1.000 1.000  0.453 0.620  0.983   
OrthProc                          1.000 1.000 1.000  0.453 0.620  0.983   
PermProc                          1.000 0.917 1.000  0.630 0.603  0.680   
ProcDist                          1.000 0.993 1.000  0.617 0.660  1.000   
RSA                               1.000 0.837 0.910  0.437 0.580  0.987   
RSMDiff                           1.000 0.993 1.000  0.807 0.540  0.653   
RankSim                           1.000 1.000 1.000  0.987 0.950  0.967   
SVCCA                             0.613 0.410 0.777  0.467 0.533  0.440   
2nd-Cos                           1.000 1.000 1.000  0.517 0.810  0.957   
SoftCorr                          0.500 0.957 0.993  0.113 0.787  0.893   
UnifDiff                          0.590 0.667 0.667  0.820 0.820  0.890   

Measure                                     Conformity Rate              \
Dataset            OGBN-Arxiv                          Cora               
Model                     GAT    GCN   SAGE             GAT   GCN  SAGE   
Similarity Measure                                                        
AlignCos                0.837  0.843  0.933           1.000 0.970 1.000   
CKA                     0.960  0.850  0.457           1.000 0.980 1.000   
ConcDiff                0.397  0.733  0.390           0.620 0.850 0.520   
DistCorr                0.933  0.807  0.523           1.000 1.000 1.000   
EOS                     1.000  1.000  1.000           1.000 1.000 1.000   
GULP                    1.000  0.733  1.000           1.000 1.000 1.000   
HardCorr                1.000  0.833  0.620           0.760 0.910 0.990   
IMD                     1.000  0.550  1.000           1.000 1.000 1.000   
Jaccard                 1.000  0.973  0.987           1.000 1.000 1.000   
LinReg                  1.000  0.993  1.000           1.000 1.000 1.000   
MagDiff                 0.633  0.060  0.290           0.920 0.630 0.720   
ShapeMet                0.993  0.967  0.993           1.000 1.000 1.000   
OrthProc                0.993  0.967  0.993           1.000 1.000 1.000   
PermProc                1.000  0.243  0.883           1.000 0.910 1.000   
ProcDist                1.000  0.833  0.853           1.000 0.990 1.000   
RSA                     0.940  0.523  0.973           1.000 0.900 0.970   
RSMDiff                 1.000  0.850  0.933           1.000 0.990 1.000   
RankSim                 1.000  0.993  0.987           1.000 1.000 1.000   
SVCCA                   0.327 -0.173 -0.097           0.810 0.630 0.800   
2nd-Cos                 1.000  0.967  1.000           1.000 1.000 1.000   
SoftCorr                1.000  0.913  0.623           0.780 0.950 0.990   
UnifDiff                0.677  0.327 -0.447           0.690 0.700 0.700   

Measure                                                       
Dataset            Flickr          

### Results of Output Correlations

In [39]:
def get_output_correlation_table(datasets, corr_func="spearmanr", acc_test = False):
    
    full_csvs = os.listdir(EXPERIMENT_RESULTS_PATH)

    dfs = []
    for dataset in datasets:
        for f in full_csvs:
            if "output" in f and dataset in f:
                df = pd.read_csv(os.path.join(EXPERIMENT_RESULTS_PATH, f))
                df = df.loc[df.loc[:,"quality_measure"] == corr_func]
                data = df.loc[:, ["similarity_measure", "functional_similarity_measure", "corr", "architecture"]]
                data = data.rename(COLUMN_NAME_DICT, axis="columns")
                data["Dataset"] = dataset
                dfs.append(data.iloc[:])

    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="corr", aggfunc="mean")
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    df_res = df_res.rename(PIVOT_COL_DICT, axis="columns")
    if acc_test: 
        return df_res.loc[:,"AbsoluteAccDiff"]
        
    return df_res.loc[:,["Disagreement", "JSD"]]

In [43]:
output_corr_df = get_output_correlation_table(["cora"])
styled = pd.io.formats.style.Styler(
    output_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("cora_results_output_correlation_pgnn.tex", **LATEX_FORMAT_DICT)
output_corr_df

Measure            Disagreement                         JSD                \
Dataset                    Cora                        Cora                 
Model                       GAT    GCN   SAGE   PGNN    GAT    GCN   SAGE   
Similarity Measure                                                          
AlignCos                 -0.051 -0.084  0.271  0.206  0.174  0.017  0.384   
CKA                       0.445  0.363  0.001  0.137  0.535  0.435 -0.031   
ConcDiff                  0.030 -0.038 -0.097 -0.066  0.014  0.032  0.041   
DistCorr                  0.456  0.529 -0.078  0.284  0.601  0.597  0.053   
EOS                      -0.088  0.127  0.023  0.294 -0.036  0.168  0.224   
GULP                     -0.129 -0.165 -0.090  0.334 -0.042 -0.096  0.182   
HardCorr                  0.261  0.454  0.104  0.450  0.516  0.479  0.162   
IMD                      -0.150  0.180 -0.103 -0.135 -0.086  0.213 -0.404   
Jaccard                   0.120  0.436  0.333  0.296  0.379  0.441  0.453   
LinReg                   -0.062  0.574  0.090  0.228  0.190  0.580  0.331   
MagDiff                  -0.069 -0.073 -0.253  0.250 -0.145  0.004  0.063   
ShapeMet                  0.053  0.616  0.156  0.312  0.282  0.695  0.272   
OrthProc                  0.053  0.616  0.156  0.312  0.282  0.695  0.272   
PWCCA                    -0.138  0.081  0.360  0.212  0.015  0.080  0.267   
PermProc                 -0.138  0.235  0.328  0.422  0.133  0.289  0.096   
ProcDist                 -0.175  0.450  0.237  0.427  0.046  0.516  0.085   
RSA                      -0.018  0.055  0.328  0.479  0.200  0.117  0.133   
RSMDiff                  -0.150  0.050  0.137  0.345 -0.072  0.115 -0.011   
RankSim                   0.442  0.343  0.227  0.303  0.505  0.363  0.561   
SVCCA                     0.166 -0.057  0.045  0.004  0.261 -0.085  0.018   
2nd-Cos                   0.299  0.399  0.252  0.153  0.529  0.429  0.536   
SoftCorr                  0.069  0.485  0.057  0.405  0.215  0.534 -0.013   
UnifDiff                 -0.146  0.067  0.161  0.276 -0.114  0.133  0.270   

Measure                    
Dataset                    
Model                PGNN  
Similarity Measure         
AlignCos           -0.143  
CKA                -0.327  
ConcDiff            0.205  
DistCorr           -0.362  
EOS                -0.225  
GULP                0.100  
HardCorr           -0.174  
IMD                 0.104  
Jaccard             0.051  
LinReg             -0.267  
MagDiff             0.101  
ShapeMet           -0.251  
OrthProc           -0.251  
PWCCA              -0.270  
PermProc            0.006  
ProcDist            0.012  
RSA                 0.106  
RSMDiff             0.006  
RankSim            -0.042  
SVCCA              -0.027  
2nd-Cos            -0.240  
SoftCorr           -0.168  
UnifDiff           -0.220

Measure            Disagreement                         JSD                \
Dataset                    Cora                        Cora                 
Model                       GAT    GCN   SAGE   PGNN    GAT    GCN   SAGE   
Similarity Measure                                                          
AlignCos                 -0.051 -0.084  0.271  0.206  0.174  0.017  0.384   
CKA                       0.445  0.363  0.001  0.137  0.535  0.435 -0.031   
ConcDiff                  0.030 -0.038 -0.097 -0.066  0.014  0.032  0.041   
DistCorr                  0.456  0.529 -0.078  0.284  0.601  0.597  0.053   
EOS                      -0.088  0.127  0.023  0.294 -0.036  0.168  0.224   
GULP                     -0.129 -0.165 -0.090  0.334 -0.042 -0.096  0.182   
HardCorr                  0.261  0.454  0.104  0.450  0.516  0.479  0.162   
IMD                      -0.150  0.180 -0.103 -0.135 -0.086  0.213 -0.404   
Jaccard                   0.120  0.436  0.333  0.296  0.379  0.441  0.453   
LinReg                   -0.062  0.574  0.090  0.228  0.190  0.580  0.331   
MagDiff                  -0.069 -0.073 -0.253  0.250 -0.145  0.004  0.063   
ShapeMet                  0.053  0.616  0.156  0.312  0.282  0.695  0.272   
OrthProc                  0.053  0.616  0.156  0.312  0.282  0.695  0.272   
PWCCA                    -0.138  0.081  0.360  0.212  0.015  0.080  0.267   
PermProc                 -0.138  0.235  0.328  0.422  0.133  0.289  0.096   
ProcDist                 -0.175  0.450  0.237  0.427  0.046  0.516  0.085   
RSA                      -0.018  0.055  0.328  0.479  0.200  0.117  0.133   
RSMDiff                  -0.150  0.050  0.137  0.345 -0.072  0.115 -0.011   
RankSim                   0.442  0.343  0.227  0.303  0.505  0.363  0.561   
SVCCA                     0.166 -0.057  0.045  0.004  0.261 -0.085  0.018   
2nd-Cos                   0.299  0.399  0.252  0.153  0.529  0.429  0.536   
SoftCorr                  0.069  0.485  0.057  0.405  0.215  0.534 -0.013   
UnifDiff                 -0.146  0.067  0.161  0.276 -0.114  0.133  0.270   

Measure                    
Dataset                    
Model                PGNN  
Similarity Measure         
AlignCos           -0.143  
CKA                -0.327  
ConcDiff            0.205  
DistCorr           -0.362  
EOS                -0.225  
GULP                0.100  
HardCorr           -0.174  
IMD                 0.104  
Jaccard             0.051  
LinReg             -0.267  
MagDiff             0.101  
ShapeMet           -0.251  
OrthProc           -0.251  
PWCCA              -0.270  
PermProc            0.006  
ProcDist            0.012  
RSA                 0.106  
RSMDiff             0.006  
RankSim            -0.042  
SVCCA              -0.027  
2nd-Cos            -0.240  
SoftCorr           -0.168  
UnifDiff           -0.220

In [70]:
output_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    output_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_output_correlation.tex", **LATEX_FORMAT_DICT)
output_corr_df

Measure            Disagreement                                                \
Dataset                    Cora               Flickr               OGBN-Arxiv   
Model                       GAT    GCN   SAGE    GAT    GCN   SAGE        GAT   
Similarity Measure                                                              
AlignCos                 -0.051 -0.084  0.271 -0.083  0.147  0.374      0.166   
CKA                       0.445  0.363  0.001  0.061 -0.211  0.534      0.227   
ConcDiff                  0.030 -0.038 -0.097  0.029 -0.214 -0.043     -0.160   
DistCorr                  0.456  0.529 -0.078 -0.031  0.172  0.395      0.204   
EOS                      -0.088  0.127  0.023  0.231  0.025  0.334      0.119   
GULP                     -0.129 -0.165 -0.090 -0.006  0.029  0.334      0.099   
HardCorr                  0.261  0.454  0.104 -0.093  0.395  0.458      0.239   
IMD                      -0.165  0.200 -0.107 -0.029 -0.103  0.344      0.002   
Jaccard                   0.120  0.436  0.333 -0.004  0.037  0.419      0.219   
LinReg                   -0.062  0.574  0.090  0.063  0.119  0.457      0.221   
MagDiff                  -0.069 -0.073 -0.253 -0.198  0.056  0.065      0.216   
ShapeMet                  0.053  0.616  0.156  0.034  0.193  0.573      0.268   
OrthProc                  0.053  0.616  0.156  0.034  0.192  0.573      0.268   
PermProc                 -0.138  0.235  0.328 -0.272  0.178 -0.097      0.221   
ProcDist                 -0.175  0.450  0.237 -0.265 -0.134 -0.148      0.303   
RSA                      -0.018  0.055  0.328  0.063  0.348  0.586      0.454   
RSMDiff                  -0.150  0.050  0.137 -0.261 -0.297 -0.003      0.356   
RankSim                   0.442  0.343  0.227  0.047  0.055  0.237      0.096   
SVCCA                     0.166 -0.057  0.045  0.028 -0.269  0.214     -0.146   
2nd-Cos                   0.299  0.399  0.252 -0.103  0.301  0.143      0.064   
SoftCorr                  0.069  0.485  0.057  0.017  0.390  0.486      0.353   
UnifDiff                 -0.146  0.067  0.161  0.182 -0.340  0.038      0.060   

Measure                             JSD                                     \
Dataset                            Cora               Flickr                 
Model                 GCN   SAGE    GAT    GCN   SAGE    GAT    GCN   SAGE   
Similarity Measure                                                           
AlignCos           -0.098  0.001  0.174  0.017  0.384 -0.013  0.306  0.443   
CKA                 0.029 -0.041  0.535  0.435 -0.031  0.166  0.029  0.576   
ConcDiff           -0.254  0.072  0.014  0.032  0.041  0.028 -0.172 -0.034   
DistCorr            0.110  0.081  0.601  0.597  0.053  0.031  0.456  0.431   
EOS                -0.255 -0.023 -0.036  0.168  0.224  0.114 -0.027  0.385   
GULP                0.154 -0.037 -0.042 -0.096  0.182  0.128  0.149  0.383   
HardCorr            0.020 -0.241  0.516  0.479  0.162  0.095  0.534  0.505   
IMD                -0.109 -0.020 -0.109  0.240 -0.396  0.033 -0.084  0.307   
Jaccard             0.142  0.012  0.379  0.441  0.453  0.107  0.331  0.421   
LinReg             -0.099 -0.188  0.190  0.580  0.331  0.176  0.152  0.477   
MagDiff            -0.252 -0.195 -0.145  0.004  0.063 -0.262  0.032  0.056   
ShapeMet            0.018 -0.165  0.282  0.695  0.272  0.130  0.432  0.626   
OrthProc            0.018 -0.165  0.282  0.695  0.272  0.130  0.432  0.626   
PermProc            0.250 -0.412  0.133  0.289  0.096 -0.424  0.289 -0.099   
ProcDist            0.257 -0.382  0.046  0.516  0.085 -0.379  0.023 -0.180   
RSA                 0.170  0.155  0.200  0.117  0.133  0.144  0.522  0.630   
RSMDiff            -0.141  0.090 -0.072  0.115 -0.011 -0.367 -0.216 -0.042   
RankSim             0.027  0.274  0.505  0.363  0.561  0.213  0.328  0.297   
SVCCA               0.085  0.113  0.261 -0.085  0.018  0.130 -0.036  0.232   
2nd-Cos             0.042  0.265  0.529  0.429  0.536  0.068  0.467  0.154   
SoftCorr            0

In [44]:
acc_corr_df = get_output_correlation_table(["cora"], acc_test=True)
styled = pd.io.formats.style.Styler(
    acc_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("cora_results_accuracy_correlation.tex", **LATEX_FORMAT_DICT)
acc_corr_df

Dataset              Cora                     
Model                 GAT    GCN   SAGE   PGNN
Similarity Measure                            
AlignCos           -0.289 -0.332  0.132  0.050
CKA                 0.023  0.163 -0.172 -0.009
ConcDiff           -0.197  0.154 -0.252  0.354
DistCorr            0.026  0.008 -0.175  0.153
EOS                -0.049 -0.236  0.080  0.272
GULP               -0.121 -0.435  0.081  0.276
HardCorr           -0.143  0.106 -0.109  0.322
IMD                -0.023 -0.114 -0.005 -0.157
Jaccard            -0.160  0.023 -0.111  0.079
LinReg             -0.109  0.058 -0.207  0.061
MagDiff            -0.192  0.077 -0.133  0.357
ShapeMet           -0.118  0.147 -0.292  0.202
OrthProc           -0.118  0.147 -0.292  0.202
PWCCA              -0.261 -0.050  0.068  0.085
PermProc           -0.261 -0.118  0.181  0.475
ProcDist           -0.192  0.083  0.006  0.475
RSA                -0.279 -0.271  0.043  0.375
RSMDiff            -0.141 -0.189  0.074  0.448
RankSim             0.321 -0.004 -0.103  0.068
SVCCA              -0.186 -0.015 -0.126 -0.015
2nd-Cos            -0.109 -0.100  0.043  0.150
SoftCorr            0.026  0.183 -0.053  0.346
UnifDiff           -0.088 -0.103 -0.037 -0.012

In [71]:
acc_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"], acc_test=True)
styled = pd.io.formats.style.Styler(
    acc_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_accuracy_correlation.tex", **LATEX_FORMAT_DICT)
acc_corr_df

Dataset              Cora               Flickr               OGBN-Arxiv  \
Model                 GAT    GCN   SAGE    GAT    GCN   SAGE        GAT   
Similarity Measure                                                        
AlignCos           -0.369 -0.188  0.154  0.088  0.173  0.388     -0.185   
CKA                -0.080 -0.310 -0.134 -0.095 -0.150  0.372     -0.129   
ConcDiff           -0.098 -0.021 -0.213  0.078 -0.110 -0.060     -0.191   
DistCorr           -0.112 -0.299 -0.126 -0.041  0.200  0.360     -0.127   
EOS                -0.002  0.290  0.085  0.399  0.227  0.224      0.048   
GULP               -0.037  0.245  0.085 -0.087  0.041  0.215      0.052   
HardCorr           -0.261  0.030 -0.116 -0.145  0.177  0.323      0.064   
IMD                -0.056 -0.118  0.038 -0.091 -0.096  0.370     -0.148   
Jaccard            -0.226 -0.042 -0.100 -0.057  0.079  0.074     -0.145   
LinReg             -0.205  0.009 -0.206 -0.115 -0.228  0.277      0.049   
MagDiff            -0.073  0.051 -0.106  0.101  0.178 -0.006     -0.101   
ShapeMet           -0.237 -0.273 -0.263 -0.093  0.187  0.427     -0.175   
OrthProc           -0.237 -0.273 -0.263 -0.093  0.188  0.427     -0.175   
PermProc           -0.317  0.058  0.166  0.094  0.349 -0.041     -0.095   
ProcDist           -0.259 -0.047 -0.008  0.090  0.007  0.013     -0.219   
RSA                -0.331 -0.143  0.064  0.025  0.332  0.300     -0.241   
RSMDiff            -0.141 -0.189  0.074  0.126 -0.184 -0.161      0.019   
RankSim             0.035 -0.041 -0.093 -0.049  0.082  0.144     -0.312   
SVCCA              -0.122 -0.039 -0.098 -0.101 -0.210 -0.119      0.119   
2nd-Cos            -0.207 -0.130  0.053 -0.000  0.393 -0.089     -0.210   
SoftCorr           -0.092 -0.130 -0.061 -0.200  0.116  0.308      0.025   
UnifDiff            0.006 -0.125 -0.033  0.067 -0.177  0.007     -0.158   

Dataset                           
Model                 GCN   SAGE  
Similarity Measure                
AlignCos           -0.069  0.201  
CKA                 0.040  0.056  
ConcDiff           -0.247 -0.004  
DistCorr            0.138  0.010  
EOS                -0.050  0.010  
GULP               -0.185 -0.013  
HardCorr            0.132 -0.040  
IMD                -0.223 -0.076  
Jaccard            -0.275 -0.132  
LinReg             -0.229 -0.021  
MagDiff            -0.214  0.033  
ShapeMet           -0.126 -0.065  
OrthProc           -0.126 -0.065  
PermProc           -0.052  0.379  
ProcDist           -0.238  0.349  
RSA                 0.560  0.171  
RSMDiff            -0.046 -0.194  
RankSim            -0.257 -0.004  
SVCCA              -0.179 -0.034  
2nd-Cos            -0.415 -0.093  
SoftCorr            0.067 -0.016  
UnifDiff           -0.175 -0.041

### CD Plots

In [21]:
# autorank package needed for creation of CD plots 
from autorank import autorank, plot_stats, create_report, latex_table
from autorank._util import *
from matplotlib import pyplot as plt

In [22]:
def get_figsize(columnwidth, wf=0.5, hf=(5. ** 0.5 - 1.0) / 2.0):
    """ Credit: https://stackoverflow.com/a/31527287
    Parameters:
      - wf [float]:  width fraction in columnwidth units
      - hf [float]:  height fraction in columnwidth units.
                     Set by default to golden ratio.
      - columnwidth [float]: width of the column in latex. Get this from LaTeX
                             using \showthe\columnwidth
    Returns:  [fig_width,fig_height]: that should be given to matplotlib
    """
    fig_width_pt = columnwidth * wf
    inches_per_pt = 1.0 / 72.27  # Convert pt to inch
    fig_width = fig_width_pt * inches_per_pt  # width in inches
    fig_height = fig_width * hf  # height in inches
    return fig_width, fig_height

PLOTS_BASE_WIDTH = 433.62  # pt
PLOTS_CD_WIDTH, PLOTS_CD_HEIGHT = get_figsize(PLOTS_BASE_WIDTH, wf=1.5)

In [23]:
def get_autorank_df(experiments=EXPERIMENTS, datasets=DATASETS, quality_measure = "violation_rate"):
    dfs = []
    for experiment in experiments:
        df = get_agg_pivot_table(experiment, datasets)        
        if experiment == "layer_test":
            dfs.append(df.loc[:,"Spearman Correlation"].dropna())
        else:
            dfs.append(df.loc[:,"AUPRC"].dropna())
    df_out = get_output_correlation_table(datasets).dropna()
    dfs.append(df_out.loc[:,"JSD"])
    df_acc = get_output_correlation_table(datasets, acc_test=True).dropna()
    dfs.append(df_acc)
    return pd.concat(dfs, axis=1)


In [24]:
get_autorank_df(datasets=["cora", "flickr", "ogbn-arxiv"])

Dataset             Cora             Flickr             OGBN-Arxiv        \
Model                GAT   GCN  SAGE    GAT   GCN  SAGE        GAT   GCN   
Similarity Measure                                                         
AlignCos           0.278 0.289 0.287  0.183 0.513 0.275      0.459 0.467   
CKA                0.274 0.273 0.274  0.141 0.445 0.415      0.728 0.846   
ConcDiff           0.212 0.131 0.198  0.156 0.218 0.235      0.505 0.518   
DistCorr           0.274 0.280 0.287  0.116 0.491 0.308      0.722 0.804   
EOS                0.132 0.160 0.168  0.123 0.268 0.284      0.222 0.170   
GULP               0.133 0.160 0.168  0.118 0.106 0.285      0.197 0.194   
HardCorr           0.270 0.272 0.273  0.191 0.389 0.332      0.334 0.520   
Jaccard            0.274 0.273 0.273  0.131 0.326 0.296      0.385 0.565   
LinReg             0.286 0.282 0.284  0.125 0.130 0.326      0.297 0.279   
MagDiff            0.185 0.119 0.134  0.110 0.410 0.273      0.172 0.270   
ShapeMet           0.273 0.273 0.276  0.143 0.537 0.313      0.454 0.494   
OrthProc           0.273 0.273 0.276  0.143 0.537 0.313      0.454 0.494   
PermProc           0.277 0.288 0.245  0.115 0.474 0.427      0.524 0.225   
ProcDist           0.280 0.286 0.290  0.115 0.456 0.394      0.624 0.425   
RSA                0.274 0.291 0.275  0.201 0.404 0.278      0.319 0.527   
RankSim            0.299 0.308 0.270  0.167 0.305 0.318      0.410 0.643   
SVCCA              0.272 0.223 0.193  0.137 0.237 0.408      0.658 0.869   
2nd-Cos            0.346 0.338 0.370  0.216 0.326 0.279      0.596 0.745   
SoftCorr           0.272 0.272 0.274  0.174 0.298 0.361      0.332 0.491   
UnifDiff           0.165 0.158 0.292  0.207 0.341 0.419      0.205 0.324   

Dataset                   Cora  ... OGBN-Arxiv   Cora               Flickr  \
Model               SAGE   GAT  ...       SAGE    GAT    GCN   SAGE    GAT   
Similarity Measure              ...                                          
AlignCos           0.464 1.000  ...      0.050 -0.369 -0.188  0.154  0.088   
CKA                0.800 1.000  ...     -0.021 -0.080 -0.310 -0.134 -0.095   
ConcDiff           0.543 0.403  ...      0.024 -0.098 -0.021 -0.213  0.078   
DistCorr           0.822 1.000  ...      0.122 -0.112 -0.299 -0.126 -0.041   
EOS                0.276 1.000  ...      0.115 -0.002  0.290  0.085  0.399   
GULP               0.277 1.000  ...      0.106 -0.037  0.245  0.085 -0.087   
HardCorr           0.515 0.540  ...     -0.276 -0.261  0.030 -0.116 -0.145   
Jaccard            0.283 1.000  ...      0.090 -0.226 -0.042 -0.100 -0.057   
LinReg             0.367 1.000  ...     -0.166 -0.205  0.009 -0.206 -0.115   
MagDiff            0.199 0.893  ...     -0.127 -0.073  0.051 -0.106  0.101   
ShapeMet           0.449 1.000  ...     -0.152 -0.237 -0.273 -0.263 -0.093   
OrthProc           0.449 1.000  ...     -0.152 -0.237 -0.273 -0.263 -0.093   
PermProc           0.257 1.000  ...     -0.552 -0.317  0.058  0.166  0.094   
ProcDist           0.480 1.000  ...     -0.503 -0.259 -0.047 -0.008  0.090   
RSA                0.284 1.000  ...      0.140 -0.331 -0.143  0.064  0.025   
RankSim            0.354 1.000  ...      0.358  0.035 -0.041 -0.093 -0.049   
SVCCA              0.723 0.613  ...      0.163 -0.122 -0.039 -0.098 -0.101   
2nd-Cos            0.710 1.000  ...      0.336 -0.207 -0.130  0.053 -0.000   
SoftCorr           0.531 0.500  ...     -0.284 -0.092 -0.130 -0.061 -0.200   
UnifDiff           0.244 0.590  ...      0.117  0.006 -0.125 -0.033  0.067   

Dataset                          OGBN-Arxiv                
Model                 GCN   SAGE        GAT    GCN   SAGE  
Similarity Measure                                         
AlignCos            0.173  0.388     -0.185 -0.069  0.201  
CKA                -0.150  0.372     -0.129  0.040  0.056  
ConcDiff           -0.110 -0.060     -0.191 -0.247 -0.004  
DistCorr            0.200  0.360     -0.127  0.138  0.010  
EOS                 0.227  0.224    

In [32]:
res_df = get_autorank_df().T.reset_index(drop=True)
result = autorank(res_df, alpha=0.05, verbose=False)
cd_diagram(result, False, None, PLOTS_CD_WIDTH)
fig = plt.gcf()
fig.set_size_inches(PLOTS_CD_WIDTH, PLOTS_CD_HEIGHT)
plt.savefig(f"cd_plot_graphs.png", bbox_inches="tight")
plt.close()

C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:432: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(sorted_ranks[i] - sorted_ranks[j]) <= critical_difference:
C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:511: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  plot_line([(rankpos(sorted_ranks[i]), cline),
C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:512: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To ac

In [29]:
get_autorank_df().T.reset_index(drop=True).shape

(54, 20)